Imports and setup:

In [ ]:
from src.config import get_config
from src.data_loader import load_and_prepare_data
from src.returns import compute_log_returns
from src.correlation import (
    average_pairwise_correlation,
    pca_eigenvalue_share
)
from src.regimes import classify_regimes
from src.portfolios import (
    equal_weight_portfolio,
    mean_variance_portfolio,
    risk_parity_portfolio,
    volatility_targeted_portfolio,
    portfolio_returns
)
from src.risk import (
    sharpe_ratio,
    max_drawdown,
    volatility
)

import pandas as pd
import matplotlib.pyplot as plt

Load Config & Data:

In [ ]:
config = get_config()

prices = load_and_prepare_data(
    config["tickers"],
    config["start_date"],
    config["end_date"]
)

prices.head()

Price Overview Plot:

In [ ]:
prices.plot(title="Asset Prices")
plt.show()

Compute Returns:

In [ ]:
returns = compute_log_returns(prices)
returns.describe()

Return Distributions:

In [ ]:
returns.hist(bins=50, figsize=(12, 8))
plt.suptitle("Return Distributions")
plt.show()

Correlation Stress Metrics:

In [ ]:
avg_corr = average_pairwise_correlation(
    returns,
    config["rolling_window"]
)

eigen_share = pca_eigenvalue_share(
    returns,
    config["rolling_window"]
)

Correlation Stress Plots:

In [ ]:
avg_corr.plot(title="Average Pairwise Correlation")
plt.show()

eigen_share.plot(title="First PCA Eigenvalue Share")
plt.show()

Regime Classification:

In [ ]:
regimes = classify_regimes(
    avg_corr,
    config["low_regime_quantile"],
    config["high_regime_quantile"]
)

regimes.value_counts()

Portfolio Construction:

In [ ]:
ew_weights = equal_weight_portfolio(returns)
mv_weights = mean_variance_portfolio(
    returns,
    config["risk_aversion"]
)
rp_weights = risk_parity_portfolio(returns)
vt_weights = volatility_targeted_portfolio(
    returns,
    config["vol_target"],
    config["trading_days"]
)

pd.DataFrame({
    "Equal Weight": ew_weights,
    "Mean Variance": mv_weights,
    "Risk Parity": rp_weights,
    "Vol Targeted": vt_weights
})

Portfolio Returns:

In [ ]:
ew_ret = portfolio_returns(returns, ew_weights)
mv_ret = portfolio_returns(returns, mv_weights)
rp_ret = portfolio_returns(returns, rp_weights)
vt_ret = portfolio_returns(returns, vt_weights)

Cumulative Performance Plot:

In [ ]:
cumulative = pd.DataFrame({
    "Equal Weight": (1 + ew_ret).cumprod(),
    "Mean Variance": (1 + mv_ret).cumprod(),
    "Risk Parity": (1 + rp_ret).cumprod(),
    "Vol Targeted": (1 + vt_ret).cumprod(),
})

cumulative.plot(title="Cumulative Portfolio Performance")
plt.show()

Risk & Performance Summary:

In [ ]:
results = pd.DataFrame({
    "Equal Weight": {
        "Sharpe": sharpe_ratio(ew_ret),
        "Max Drawdown": max_drawdown(ew_ret),
        "Volatility": volatility(ew_ret),
    },
    "Mean Variance": {
        "Sharpe": sharpe_ratio(mv_ret),
        "Max Drawdown": max_drawdown(mv_ret),
        "Volatility": volatility(mv_ret),
    },
    "Risk Parity": {
        "Sharpe": sharpe_ratio(rp_ret),
        "Max Drawdown": max_drawdown(rp_ret),
        "Volatility": volatility(rp_ret),
    },
    "Vol Targeted": {
        "Sharpe": sharpe_ratio(vt_ret),
        "Max Drawdown": max_drawdown(vt_ret),
        "Volatility": volatility(vt_ret),
    },
})

results.T